In [1]:
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/')

In [2]:
from venture.shortcuts import *
import venture.lite.builtin as bp
import venture.lite.types as t
import venture.lite.psp as psp
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt
from util import *
from collections import defaultdict

%matplotlib inline

In [3]:
v=make_lite_venture_script_ripl()
v.bind_foreign_sp("compute_distance", bp.deterministic_typed(compute_distance2, 
                  [t.NumberType()]*3,t.NumberType()))
v.bind_foreign_sp("compute_azimuth", bp.deterministic_typed(compute_azimuth2, 
                  [t.NumberType()]*3,t.NumberType()))

class BoundedExponOutputPSP(psp.RandomPSP):
  # TODO don't need to be class methods
  def simulateNumeric(self,mu,theta,gamma):
    while True:
        x = mu + sp.expon.rvs(scale=1.0/theta)
        if x < gamma:
            return x
  def logDensityNumeric(self,x,mu,theta,gamma):
    nontailmass = sp.expon.logcdf(gamma - mu,scale=1.0/theta)
    return sp.expon.logpdf(x - mu,scale=1.0/theta) - nontailmass

  def simulate(self,args): return self.simulateNumeric(*args.operandValues())
  def logDensity(self,x,args): return self.logDensityNumeric(x,*args.operandValues())

  def gradientOfLogDensity(self,x,args):
    mu,theta,gamma = args.operandValues()
    nontailmass = sp.expon.cdf(gamma - mu,scale=1.0/theta)
    gradX = -theta
    gradTheta = 1.0 / nontailmass / theta - (x - mu)
    return (gradX,[gradTheta])

  def description(self,name):
    return "  %s(theta) returns a sample from an exponential distribution with rate (inverse scale) parameter theta." % name


v.bind_foreign_sp("bounded_expon", bp.typed_nr(BoundedExponOutputPSP(), [t.NumberType()]*3,t.NumberType()))

## Initialize Priors

In the next block, I initialize all parameters of a physical world description via the prior distributions given in the problem description file. I set the assumptions in my ripl layer while defining the variables locally, then set all local variables into the namedTuple physicsPriors.

In [4]:
lenst = len(STATIONS)

physPriorParamNames = ["T", "R", "lambda_e", "mu_m", "theta_m", "gamma_m", "mu_d0", "mu_d1", "mu_d2",
  "mu_t", "theta_t", "mu_z", "theta_z", "mu_s", "theta_s",
  "mu_a0", "mu_a1", "mu_a2", "sigma_a",
  "lambda_f", "mu_f", "theta_f"]

T = v.assume('T','3600.')
R = v.assume('R','6371.')
lambda_e = v.assume('lambda_e','gamma(6.0,(4*3.1415927*6371**2*3600))')
mu_m = v.assume('mu_m','3.0')
theta_m = v.assume('theta_m','4.0')
gamma_m = v.assume('gamma_m','6.0')
v.assume('mu_d_covar','matrix(list(list(13.43,-2.36,-0.0122),list(-2.36,0.452,0.000112),list(-0.0122,0.000112,0.000125)))')
v.assume('mu_a_covar','matrix(list(list(1.23, -0.227, -0.000175),list(-0.227,0.0461,0.0000245),list(-0.000175,0.0000245,0.000000302)))')

v.assume('mu_d','mem( proc(k) { multivariate_normal(vector(-10.4,3.26,-0.0499),mu_d_covar) })')
v.assume('mu_t','mem( proc(k) { 0.0 } )')
v.assume('theta_t','mem( proc(k) { inv_gamma(120,118) } )')
v.assume('mu_z','mem( proc(k) { 0.0 } )')
v.assume('theta_z','mem( proc(k) { inv_gamma(5.2,44) } )')
v.assume('mu_s','mem( proc(k) { 0.0 } )')
v.assume('theta_s','mem( proc(k) { inv_gamma(6.7,7.5) } )')
v.assume('mu_a','mem( proc(k) { multivariate_normal(vector(-7.3,2.03,-0.00196),mu_a_covar) })')
v.assume('sigma_a','mem( proc(k) { (inv_gamma(21.1,12.6)**0.5) } )')
v.assume('lambda_f','mem( proc(k) { gamma(2.1,1 / 0.0013) } )')
v.assume('mu_f','mem( proc(k) { normal(-0.68,0.68) } )')
v.assume('theta_f','mem( proc(k) { inv_gamma(23.5,12.45) } )')

mu_d = np.zeros((lenst,3))
mu_t = np.zeros(lenst)
theta_t = np.zeros(lenst)
mu_z = np.zeros(lenst)
theta_z = np.zeros(lenst)
mu_s = np.zeros(lenst)
theta_s = np.zeros(lenst)
mu_a = np.zeros((lenst,3))
sigma_a = np.zeros(lenst)
lambda_f = np.zeros(lenst)
mu_f = np.zeros(lenst)
theta_f = np.zeros(lenst)


pp = []
for i in range(len(STATIONS)):
    mu_d[i] = v.sample('mu_d({0})'.format(i))
    mu_t[i] = v.sample('mu_t({0})'.format(i))
    theta_t[i] = v.sample('theta_t({0})'.format(i))
    mu_z[i] = v.sample('mu_z({0})'.format(i))
    theta_z[i] = v.sample('theta_z({0})'.format(i))
    mu_s[i] = v.sample('mu_s({0})'.format(i))
    theta_s[i] = v.sample('theta_s({0})'.format(i))
    mu_a[i] = v.sample('mu_a({0})'.format(i))
    sigma_a[i] = v.sample('sigma_a({0})'.format(i))
    lambda_f[i] = v.sample('lambda_f({0})'.format(i))
    mu_f[i] = v.sample('mu_f({0})'.format(i))
    theta_f[i] = v.sample('theta_f({0})'.format(i))
    

physicsPriors = Physics(T=T, R=R, 
                        lambda_e=lambda_e,
                        mu_m=mu_m, theta_m=theta_m, gamma_m=gamma_m, 
                        mu_d0=[item[0] for item in mu_d],
                        mu_d1=[item[1] for item in mu_d],
                        mu_d2=[item[2] for item in mu_d],
                        mu_t=mu_t,
                        theta_t=theta_t,
                        mu_z=mu_z,
                        theta_z=theta_z,
                        mu_s=mu_s,
                        theta_s=theta_s,
                        mu_a0=[item[0] for item in mu_a],
                        mu_a1=[item[1] for item in mu_a],
                        mu_a2=[item[2] for item in mu_a],
                        sigma_a=sigma_a,
                        lambda_f=lambda_f,
                        mu_f=mu_f,
                        theta_f=theta_f,)


##Set Model Dependencies from Observations to Background Parameters

In the next block, I set all model dependencies so that my training data will affect the background physics parameters.

In [5]:
#Number of Events in an Episode
v.assume('num_e','mem( proc(epi) {poisson(lambda_e*4*3.1415927*R**2*T)})')

#
#DO I ENCODE UNIFORM DISTRIBUTIONS OF EVENT TIME AND LOCATION?
v.assume('e_t','mem( proc(epi,i) { uniform_continuous(0,T) } )')
v.assume('e_lon','mem ( proc(epi,i) { uniform_continuous(-180,180) } )')
v.assume('sin_e_lat','mem( proc(epi,i) {uniform_continuous(-1,1)})')
#

#Event Magnitude Distribution
v.assume('e_m','mem( proc(epi,i) { bounded_expon(mu_m,1/theta_m,gamma_m) } )')
#Condition Exponential on bounds, using uniform_continuous
#v.assume('obs_func','proc(i){uniform_continuous(mu_m-e_m(epi,i),gamma_m-e_m(epi,i))}')
#v.observe('obs_func(1)','0')

#Detection Probability
v.assume('det_prob','''mem( proc(epi,i,k) {1./(1 + exp(
                                -1. * (vector_dot(mu_d(k),
                                vector(1,e_m(epi,i),compute_distance(k,e_lon(epi,i),sin_e_lat(epi,i)))))))} )
                                ''')
#Travel Time Function
v.assume('I_t','proc(dist) { -0.023 * pow(dist,2) + 10.7 * dist + 5 } ')

#Detection Time
v.assume('Lambda_t','''mem( proc(epi,i,k) {
         laplace(e_t(epi,i)+I_t(compute_distance(k,e_lon(epi,i),sin_e_lat(epi,i)))+mu_t(k),
         theta_t(k)) } )''')

#Detection Azimuth
v.assume('Lambda_z','''mem( proc(epi,i,k) { 
          laplace(compute_azimuth(k,e_lon(epi,i),sin_e_lat(epi,i)) - mu_z(k),theta_z(k)) } ) ''' )
#Maybe goes over 360? Problem?

#Slowness Function
v.assume('I_s','proc(dist) { -0.046 * dist + 10.7 } ')

#Detection Slowness
v.assume('Lambda_s','''mem( proc(epi,i,k) { laplace(I_s(
         compute_distance(k,e_lon(epi,i),sin_e_lat(epi,i)))+mu_s(k),theta_s(k)) })''')

#Detection Amplitude
v.assume('log_Lambda_a','''mem( proc(epi,i,k) {normal(vector_dot(mu_a(k),
                                vector(1,e_m(epi,i),I_t(compute_distance(k,e_lon(epi,i),sin_e_lat(epi,i))))),sigma_a(k))})''')

#False Detection Parameters
v.assume('num_Xi','mem( proc(epi,k) {poisson(lambda_f(k) * T)} )')
v.assume('Xi_t','mem( proc(epi,k,f) {uniform_continuous(0,T)})')
v.assume('Xi_z','mem( proc(epi,k,f) {uniform_continuous(0,360)})')
v.assume('Xi_s','mem( proc(epi,k,f) {uniform_continuous(I_s(180),I_s(0))})')
v.assume('Xi_a','mem( proc(epi,k,f) {student_t(1,mu_f(k),theta_f(k))})')


'<procedure>'

In [6]:
episodes = read_episodes('../short_data/training.data')

In [7]:
for i in range(len(STATIONS)):
    print v.sample('theta_z({0})'.format(i))


9.69359659075
8.03390340825
15.6677757084
4.11447034866
5.40816360728
5.10928757386
13.4993409678
4.61395569651
18.1491082675
11.9478137744


In [8]:
def check_weight():
    return
    print v.infer('particle_log_weights')[0]
    if v.infer('particle_log_weights')[0] == float('-inf'):
        raise Exception()

In [9]:
for i in range(len(episodes)):
    print 'OBSERVING EPISODE (%s)' % (i)
    v.observe('num_e(%s)'% i,'%s' % len(episodes[i].events))
    check_weight()
    for event_id,event in enumerate(episodes[i].events):
        v.observe('e_t({0},{1})'.format(i,event_id),'%s' % event.time)
        check_weight()
        v.observe('e_lon({0},{1})'.format(i,event_id),'%s' % event.lon)
        check_weight()
        v.observe('sin_e_lat({0},{1})'.format(i,event_id),'%s' % np.sin(np.radians(event.lat)))
        check_weight()
        v.observe('e_m({0},{1})'.format(i,event_id),'%s' % event.mag)
        check_weight()
    for event_id, event_dets in enumerate(episodes[i].assocs):
        for detection_id in event_dets:
            ev = episodes[i].events[event_id]
            det = episodes[i].detections[detection_id]
            v.observe('Lambda_t(%s, %s, %s)' % (i,event_id, det.stanum), '%s' % det.time)
            check_weight()
            v.observe('Lambda_z(%s, %s, %s)' % (i,event_id, det.stanum),'%s' % det.azimuth)
            check_weight()
            v.observe('Lambda_s(%s, %s, %s)' % (i,event_id, det.stanum),'%s' % det.slowness)
            check_weight()
            v.observe('log_Lambda_a(%s, %s, %s)' % (i,event_id, det.stanum),'%s' % np.log(det.amp))
            check_weight()
    station_counters = defaultdict(int)
    for false_det_id in range(len(sum(episodes[i].assocs,[])),len(episodes[i].detections)):
        false_det = episodes[i].detections[false_det_id]
        station_counters[false_det.stanum] += 1
        v.observe('Xi_a(%s, %s, %s)' % (i,det.stanum,false_det_id),'%s' % np.log(det.amp))
        check_weight()
    for stanum in range(len(STATIONS)):
        v.observe('num_Xi(%s, %s)' % (i,stanum), '%s' % station_counters[stanum])
        check_weight()
        
#v.infer('bogo_possibilize(default, all,1)')

print 'done'

OBSERVING EPISODE (0)
OBSERVING EPISODE (1)
OBSERVING EPISODE (2)
OBSERVING EPISODE (3)
OBSERVING EPISODE (4)
OBSERVING EPISODE (5)
OBSERVING EPISODE (6)
OBSERVING EPISODE (7)
OBSERVING EPISODE (8)
OBSERVING EPISODE (9)
OBSERVING EPISODE (10)
OBSERVING EPISODE (11)
OBSERVING EPISODE (12)
OBSERVING EPISODE (13)
OBSERVING EPISODE (14)
OBSERVING EPISODE (15)
OBSERVING EPISODE (16)
OBSERVING EPISODE (17)
OBSERVING EPISODE (18)
OBSERVING EPISODE (19)
OBSERVING EPISODE (20)
OBSERVING EPISODE (21)
OBSERVING EPISODE (22)
OBSERVING EPISODE (23)
OBSERVING EPISODE (24)
OBSERVING EPISODE (25)
OBSERVING EPISODE (26)
OBSERVING EPISODE (27)
OBSERVING EPISODE (28)
OBSERVING EPISODE (29)
OBSERVING EPISODE (30)
OBSERVING EPISODE (31)
OBSERVING EPISODE (32)
OBSERVING EPISODE (33)
OBSERVING EPISODE (34)
OBSERVING EPISODE (35)
OBSERVING EPISODE (36)
OBSERVING EPISODE (37)
OBSERVING EPISODE (38)
OBSERVING EPISODE (39)
OBSERVING EPISODE (40)
OBSERVING EPISODE (41)
OBSERVING EPISODE (42)
OBSERVING EPISODE (43

In [10]:
print v.infer('global_posterior')
print v.infer('global_likelihood')
v.infer('default_markov_chain(300)')

[-72102.453971690542]
[-72216.423455091935]


[]

In [11]:
data = []
for i in range(50):
    v.infer('default_markov_chain(10)')
    data.append([v.sample('theta_s({0})'.format(k)) for k in range(len(STATIONS))])
    

In [12]:
for i in range(len(STATIONS)):
    print 'Theta_s{0}'.format(i), v.sample('theta_s({0})'.format(i))
for i in range(len(STATIONS)):
    print 'Lambda_f{0}'.format(i), v.sample('lambda_f({0})'.format(i))
    
print v.sample('lambda_e')

Theta_s0 2.39361104927
Theta_s1 2.38384125135
Theta_s2 0.700419633498
Theta_s3 0.905826016529
Theta_s4 1.86466651872
Theta_s5 1.12714366214
Theta_s6 3.11513741693
Theta_s7 1.47150826776
Theta_s8 2.10294084153
Theta_s9 1.05983519681
Lambda_f0 0.00429758346796
Lambda_f1 0.00463106177004
Lambda_f2 0.00294923537358
Lambda_f3 0.00186162362694
Lambda_f4 0.0013110637409
Lambda_f5 0.00114489697842
Lambda_f6 0.00164150732338
Lambda_f7 0.00106116563441
Lambda_f8 0.00201210196231
Lambda_f9 0.00264079678921
2.00151870686e-12


In [15]:
f = open('data.txt','w')
import json
json.dump(data,f)
f.close()

In [13]:
print data

[[2.0066058024412765, 1.5931299001429224, 0.7693168449524597, 1.0078600440585155, 1.4000072111157424, 1.2535480066964282, 1.5033461713319316, 1.3063303878057437, 1.3351387509620605, 1.1673036954295641], [2.0066058024412765, 1.5931299001429224, 0.7693168449524597, 1.0078600440585155, 1.4000072111157424, 1.2535480066964282, 1.5033461713319316, 1.3063303878057437, 1.3351387509620605, 1.1673036954295641], [2.0066058024412765, 1.5931299001429224, 0.7693168449524597, 1.0078600440585155, 1.4000072111157424, 1.2535480066964282, 1.5033461713319316, 1.3063303878057437, 1.3351387509620605, 1.1673036954295641], [2.0066058024412765, 1.5931299001429224, 0.7693168449524597, 1.0078600440585155, 1.4000072111157424, 1.2535480066964282, 1.5033461713319316, 1.3063303878057437, 1.3351387509620605, 1.1673036954295641], [2.0066058024412765, 1.5931299001429224, 0.7693168449524597, 1.0078600440585155, 1.4000072111157424, 1.2535480066964282, 1.5033461713319316, 1.3457284981260702, 1.3351387509620605, 1.16730369